In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# --- CONFIGURAÇÃO DOS CAMINHOS ---
base_dir = r"C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoTrade"

# Lista completa de modelos para garantir que todos apareçam
resultados_dirs = {
    'LSTM Macro': os.path.join(base_dir, 'FeaturesMacroResultados'),
    'LSTM Micro': os.path.join(base_dir, 'FeaturesMicroResultados'),
    'LSTM MacroMicro': os.path.join(base_dir, 'FeaturesMacroMicroResultados'),
    'Transformers Macro': os.path.join(base_dir, 'TransformersMacroResultados'),
    'Transformers Micro': os.path.join(base_dir, 'TransformersMicroResultados'),
    'Transformers MacroMicro': os.path.join(base_dir, 'TransformersMacroMicroResultados')
}

output_dir = os.path.join(base_dir, 'AnaliseComparativaFinal')
os.makedirs(output_dir, exist_ok=True)

print("="*80)
print(" "*20 + "GERANDO GRÁFICOS FINAIS AJUSTADOS")
print("="*80)

# --- 1. CARREGAMENTO DOS DADOS ---
all_results = []

for model_name, dir_path in resultados_dirs.items():
    print(f"📂 Lendo: {model_name}")
    # Identifica se é LSTM ou Transformer para pegar o arquivo correto
    if 'LSTM' in model_name:
        nome_arquivo = 'melhores_resultados_realista_features.csv'
    else:
        nome_arquivo = 'melhores_resultados_realista_transformer.csv'
        
    arquivo_path = os.path.join(dir_path, nome_arquivo)
    
    if os.path.exists(arquivo_path):
        df = pd.read_csv(arquivo_path)
        df['Modelo'] = model_name
        all_results.append(df)
    else:
        # Se o arquivo não existir, cria um DataFrame vazio para não quebrar o loop, 
        # mas avisa o usuário.
        print(f"  ⚠️ AVISO: {nome_arquivo} não encontrado em {model_name}")

if not all_results:
    raise ValueError("Nenhum dado encontrado nos caminhos especificados.")

df_completo = pd.concat(all_results, ignore_index=True)
# Remove PETR4 se necessário, conforme seu padrão anterior
df_completo = df_completo[df_completo['Ticker'] != 'PETR4.SA']

# Separa os DataFrames por arquitetura
df_lstm = df_completo[df_completo['Modelo'].str.contains('LSTM')].copy()
df_transformer = df_completo[df_completo['Modelo'].str.contains('Transformer')].copy()

# --- 2. FUNÇÃO DE PLOTAGEM GENÉRICA (AJUSTE AUTOMÁTICO) ---
def plotar_grafico_isolado(df_data, metrica, titulo, arquivo_saida, y_label, linha_ref=None, is_percent=False):
    if df_data.empty:
        print(f"⚠️ Pular gráfico {arquivo_saida}: Dados vazios.")
        return

    plt.figure(figsize=(14, 7))
    
    # Pivota os dados: Ticker nas linhas, Modelos nas colunas
    pivot_data = df_data.pivot(index='Ticker', columns='Modelo', values=metrica)
    
    # Cria o gráfico
    ax = pivot_data.plot(kind='bar', figsize=(14, 8), width=0.8, edgecolor='black', linewidth=1)
    
    plt.title(titulo, fontsize=18, fontweight='bold', pad=20)
    plt.ylabel(y_label, fontsize=12, fontweight='bold')
    plt.xlabel('Ticker', fontsize=12, fontweight='bold')
    plt.xticks(rotation=0, fontsize=11)
    plt.legend(title='Modelo', bbox_to_anchor=(1.01, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    
    # Linha de referência (ex: Sharpe = 1 ou Retorno = 0)
    if linha_ref is not None:
        plt.axhline(y=linha_ref, color='red' if linha_ref == 0 else 'green', 
                   linestyle='--', linewidth=2, alpha=0.7)

    # Ajuste dinâmico do eixo Y para caber o texto
    y_min, y_max = ax.get_ylim()
    margin = (y_max - y_min) * 0.15
    ax.set_ylim(y_min, y_max + margin)

    # Anotação dos valores (Labels)
    for container in ax.containers:
        for bar in container:
            height = bar.get_height()
            if pd.isna(height):
                continue
            
            # Formata o texto
            if is_percent:
                label_text = f'{height:.1f}%'
            else:
                label_text = f'{height:.2f}'
            
            # Define posição do texto (acima se positivo, abaixo se negativo)
            offset = (0, 5) if height >= 0 else (0, -15)
            va = 'bottom' if height >= 0 else 'top'
            
            ax.annotate(label_text,
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=offset,
                        textcoords="offset points",
                        ha='center', va=va,
                        fontsize=9, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.6, ec="none"))

    plt.tight_layout()
    path_completo = os.path.join(output_dir, arquivo_saida)
    plt.savefig(path_completo, dpi=300)
    plt.close()
    print(f"✓ Salvo: {arquivo_saida}")

# --- 3. GERAR GRÁFICOS SEPARADOS (SHARPE E RETORNO) ---

print("\n📊 Gerando gráficos LSTM...")
plotar_grafico_isolado(df_lstm, 'Sharpe Ratio', 'LSTM: Comparativo de Sharpe Ratio', 
                       'analise_lstm_sharpe.png', 'Sharpe Ratio', linha_ref=1.0)
plotar_grafico_isolado(df_lstm, 'Retorno Liquido (%)', 'LSTM: Retorno Líquido (%)', 
                       'analise_lstm_retorno.png', 'Retorno (%)', linha_ref=0.0, is_percent=True)

print("\n📊 Gerando gráficos Transformers...")
plotar_grafico_isolado(df_transformer, 'Sharpe Ratio', 'Transformers: Comparativo de Sharpe Ratio', 
                       'analise_transformers_sharpe.png', 'Sharpe Ratio', linha_ref=1.0)
plotar_grafico_isolado(df_transformer, 'Retorno Liquido (%)', 'Transformers: Retorno Líquido (%)', 
                       'analise_transformers_retorno.png', 'Retorno (%)', linha_ref=0.0, is_percent=True)


# --- 4. GRÁFICO DE VITÓRIAS CORRIGIDO (BARRAS EM VEZ DE PIZZA) ---
# O objetivo aqui é mostrar TODOS os modelos, mesmo os que tiveram 0 vitórias

print("\n📊 Gerando Gráfico de Distribuição de Vitórias (TODOS OS MODELOS)...")

# Identifica o melhor modelo para cada ticker
melhor_por_ticker = []
tickers = df_completo['Ticker'].unique()

for ticker in tickers:
    df_ticker = df_completo[df_completo['Ticker'] == ticker]
    if not df_ticker.empty:
        # Critério de vitória: Maior Sharpe Ratio
        melhor = df_ticker.loc[df_ticker['Sharpe Ratio'].idxmax()]
        melhor_por_ticker.append(melhor['Modelo'])

# Conta as vitórias
vitorias_series = pd.Series(melhor_por_ticker).value_counts()

# TRUQUE: Reindexar com a lista completa de chaves do dicionário original
# Isso força o pandas a incluir os modelos com contagem 0
todos_modelos_nomes = list(resultados_dirs.keys())
vitorias_completas = vitorias_series.reindex(todos_modelos_nomes, fill_value=0)

# Plotando
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(vitorias_completas)))

# Gráfico de barras horizontal para facilitar leitura dos nomes
bars = vitorias_completas.plot(kind='barh', color=colors, edgecolor='black', width=0.7, ax=ax)

ax.set_title('Contagem de "Vitórias" por Modelo\n(Quem teve o melhor Sharpe por Ticker)', 
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Quantidade de Tickers conquistados', fontsize=12, fontweight='bold')
ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')

# Limite do eixo X (um pouco maior que o máximo para caber o texto)
ax.set_xlim(0, vitorias_completas.max() + 1)

# Adiciona os números na frente das barras
for i, v in enumerate(vitorias_completas):
    ax.text(v + 0.1, i, str(v), color='black', fontweight='bold', va='center', fontsize=12)

plt.tight_layout()
graph_vitorias = os.path.join(output_dir, 'distribuicao_vitorias_todos_modelos.png')
plt.savefig(graph_vitorias, dpi=300)
plt.close()
print(f"✓ Salvo: {graph_vitorias} (Inclui modelos com 0 vitórias)")

print("\n" + "="*80)
print("✅ PROCESSO FINALIZADO")
print(f"📁 Verifique a pasta: {output_dir}")

                    GERANDO GRÁFICOS FINAIS AJUSTADOS
📂 Lendo: LSTM Macro
📂 Lendo: LSTM Micro
📂 Lendo: LSTM MacroMicro
📂 Lendo: Transformers Macro
📂 Lendo: Transformers Micro
📂 Lendo: Transformers MacroMicro

📊 Gerando gráficos LSTM...
✓ Salvo: analise_lstm_sharpe.png
✓ Salvo: analise_lstm_retorno.png

📊 Gerando gráficos Transformers...
✓ Salvo: analise_transformers_sharpe.png
✓ Salvo: analise_transformers_retorno.png

📊 Gerando Gráfico de Distribuição de Vitórias (TODOS OS MODELOS)...
✓ Salvo: C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoTrade\AnaliseComparativaFinal\distribuicao_vitorias_todos_modelos.png (Inclui modelos com 0 vitórias)

✅ PROCESSO FINALIZADO
📁 Verifique a pasta: C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoTrade\AnaliseComparativaFinal


<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>